In [177]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import xgboost as xgb
import random
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.calibration import CalibratedClassifierCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import missingno as msno 
import eli5
import scikitplot as skplt
from imblearn.over_sampling import SMOTE
from eli5.sklearn import PermutationImportance
warnings.filterwarnings("ignore") 

In [133]:
from sklearn.svm import SVC


In [134]:
base_final = pd.read_csv('base_final.csv')

In [135]:
base_final.columns

Index(['id', 'age', 'monthly_income', 'collateral_value', 'loan_amount',
       'collateral_debt', 'monthly_payment', 'auto_year', 'Total_value',
       'Total_monthly', 'pre_approved', 'form_completed', 'education_level',
       'verified_restriction', 'zip_code', 'gender', 'expired_debts',
       'dishonored_checks', 'protests'],
      dtype='object')

In [136]:
X = base_final.drop(['pre_approved', 'id'], axis=1).values
y = base_final.pre_approved.values
sss_1 = StratifiedShuffleSplit(n_splits=2, train_size=0.7, random_state=1234)
sss_1.get_n_splits(X, y)
for train_index, rem_index in sss.split(X, y):
    X_train, X_rem = X[train_index], X[rem_index]
    y_train, y_rem = y[train_index], y[rem_index]
    
sss_2 = StratifiedShuffleSplit(n_splits=2, train_size=0.66, random_state=1234)
sss_2.get_n_splits(X_rem, y_rem)
for val_index, test_index in sss.split(X_rem, y_rem):
    X_val, X_test = X_rem[val_index], X_rem[test_index]
    y_val, y_test = y_rem[val_index], y_rem[test_index]

In [137]:
xgboost_clf = xgb.XGBClassifier(scale_pos_weight=61.59/38.41)

In [182]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
# modelo base e hiperparametros 
model = xgb.XGBClassifier(scale_pos_weight=61.59/38.41, learning_rate =0.01, n_estimators=500)
p_grid = {'max_depth':list(range(4,12,1)),
          'gamma':[i/10.0 for i in range(0,5)],
          'min_child_weight':range(1,6,2),
          'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}
# cross validation interno (variando conjunto de validação) 
grid_search_cv = RandomizedSearchCV(model, p_grid, cv=2, n_jobs=-1, n_iter=50, verbose=1)
# cross validation externo (variando conjunto de teste)
nested_cv_scores = cross_val_score(grid_search_cv, X, y, cv=4)

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[15:15:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 2 folds for each of 50 candidates, totalling 100 fits
[15:19:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 2 folds for each of 50 candidates, totalling 100 fits
[15:23:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:log

In [183]:
nested_cv_scores

array([0.94872683, 0.94716785, 0.7033957 , 0.746535  ])

In [184]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
#
n_folds = 4 # cv folds externos
k_folds = 5 # cv folds internos
# define cv externo
cv_outer = StratifiedShuffleSplit(n_splits=n_folds)
test_acc_results = list()
# loop CV externo 
for train_i, test_i in cv_outer.split(X,y):
  # usa indices para pegar os dados efetivamente
    train_x, train_y = X[train_i], y[train_i]
    test_x, test_y = X[test_i], y[test_i]
    # modelo base
    model = xgb.XGBClassifier(scale_pos_weight=61.59/38.41, learning_rate =0.01, n_estimators=500)
    # cross validation interno (variando conjunto de validação) 
    grid_search_cv = RandomizedSearchCV(model, p_grid, cv=k_folds, scoring='accuracy', refit=True)
    result = grid_search_cv.fit(train_x, train_y)
    # retreina melhor modelo  com (treino + validação)
    best_model = result.best_estimator_
    # faz a previsão no fold de teste 
    pred_y = best_model.predict(test_x) 
    # avalia o modelo e armazena o resultado
    test_acc = accuracy_score(test_y, pred_y)
    test_acc_results.append(test_acc) 
    
    print('Acc Validação: {:.2} | Acc Teste: {:.2}'.format(result.best_score_, test_acc))
    print('Parâmetros do melhor modelo para o fold atual(externo): ', result.best_params_)

[15:39:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:39:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:39:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:40:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:42:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:42:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:42:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:42:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:45:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:48:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:48:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:48:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:48:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:52:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:52:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:52:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:56:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:56:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:56:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:56:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[15:58:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:58:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:59:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[22:43:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:43:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:44:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:44:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [138]:
xgboost_clf.fit(X_train, y_train)
y_hat = xgb_calibrated.predict_proba(X_test)

[13:45:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:45:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:45:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [139]:
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test, y_hat[:,1])
roc_auc = metrics.auc(fpr, tpr)
print('A Área sob a curva ROC é %0.3f:' %roc_auc)

A Área sob a curva ROC é 0.955:


<h3> Stacking </h3>

In [140]:
rfc = RandomForestClassifier(class_weight='balanced')
etc = ExtraTreesClassifier(class_weight='balanced')
svm = SVC(class_weight='balanced', kernel='rbf', probability=False)
xgb = xgb.XGBClassifier(scale_pos_weight=61.59/38.41)

In [141]:
xgb.fit(X_train, y_train)
rfc.fit(X_train, y_train)
etc.fit(X_train, y_train)
svm.fit(X_train, y_train)

[13:46:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


SVC(class_weight='balanced')

In [142]:
y_xgb_val = xgb.predict_proba(X_val)[:,1]
y_rfc_val = rfc.predict_proba(X_val)[:,1]
y_etc_val = etc.predict_proba(X_val)[:,1]
y_svm_val = svm.predict(X_val)

y_xgb_test = xgb.predict_proba(X_test)[:,1]
y_rfc_test = rfc.predict_proba(X_test)[:,1]
y_etc_test = etc.predict_proba(X_test)[:,1]
y_svm_test = svm.predict(X_test)

In [155]:
X_blender_val = pd.DataFrame({'xgb': y_xgb_val, 'rfc':y_rfc_val, 'etc':y_etc_val, 'svm':y_svm_val})
X_blender_test = pd.DataFrame({'xgb': y_xgb_test, 'rfc':y_rfc_test, 'etc':y_etc_test, 'svm':y_svm_test})

In [156]:
X_blender_val

,xgb,rfc,etc,svm
0,0.992185,0.95,0.91,1
1,0.100477,0.19,0.17,0
2,0.322190,0.27,0.23,0
3,0.390174,0.30,0.12,0
4,0.940184,0.95,0.94,1
...,...,...,...,...
3689,0.000178,0.01,0.00,0
3690,0.265084,0.26,0.30,0
3691,0.000483,0.00,0.00,0
3692,0.096082,0.08,0.22,0


In [157]:
X_blender_val.corr()

,xgb,rfc,etc,svm
xgb,1.000000,0.971399,0.953984,0.857704
rfc,0.971399,1.000000,0.980726,0.881904
etc,0.953984,0.980726,1.000000,0.893154
svm,0.857704,0.881904,0.893154,1.000000


In [162]:
etc.fit(X_blender_val, y_val)
rfc.fit(X_blender_val, y_val)
xgb.fit(X_blender_val, y_val)
svm.fit(X_blender_val, y_val)

[14:42:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


SVC(class_weight='balanced')

In [164]:
y_blender_pred_etc = etc.predict_proba(X_blender_test)
y_blender_pred_rfc = rfc.predict_proba(X_blender_test)
y_blender_pred_xgb = xgb.predict_proba(X_blender_test)
y_blender_pred_svm = svm.predict(X_blender_test)

In [175]:
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test, X_blender_test.mean(axis=1))
roc_auc = metrics.auc(fpr, tpr)
print('Para a baseline, a Área sob a curva ROC é %0.3f:' %roc_auc)

Para a baseline, a Área sob a curva ROC é 0.951:


In [168]:
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test, y_blender_pred_svm)
roc_auc = metrics.auc(fpr, tpr)
print('A Área sob a curva ROC é %0.3f:' %roc_auc)

A Área sob a curva ROC é 0.866:


In [161]:
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test, X_blender_test['xgb'])
roc_auc = metrics.auc(fpr, tpr)
print('A Área sob a curva ROC é %0.3f:' %roc_auc)

A Área sob a curva ROC é 0.953:


In [169]:
X_blender_test.mean(axis=1)

0      0.426463
1      0.283054
2      0.000285
3      0.002569
4      0.949266
         ...   
919    0.044736
920    0.000125
921    0.943220
922    0.322879
923    0.721073
Length: 924, dtype: float64